In [1]:
#!pip install dash

In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the SpaceX launch data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout

sites=spacex_df['Launch Site'].unique().tolist()
sites.insert(0, 'All Sites')


app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                              {'label': 'All Sites', 'value': 'All'},
                                              {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                              {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                              {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                              {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                              ],
                                              value='All',
                                              placeholder="Select a Launch Site here",
                                              searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                value=[min_payload, max_payload],
                                                marks={0: '0 kg', 2500: '2500',
                                                5000: '5000', 7500: '7500',
                                                10000: '10000'}),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value')
              )

def get_pie_chart(value):
    filtered_df = spacex_df
    if value == 'All':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == value]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        title = f"Total Success Launches for site {value}"
        fig = px.pie(filtered_df, values='class count', names='class',title=title)
        return fig        

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')]
              )

def get_scatter(value1, value2):
    filtered_df1 = spacex_df[(spacex_df['Payload Mass (kg)'] > value2[0]) & (spacex_df['Payload Mass (kg)'] < value2[1])]

    if value1 == 'All':
        fig = px.scatter(filtered_df1, x = "Payload Mass (kg)", y = 'class', \
            color = "Booster Version Category", \
            title = "Correlation between Payload and Success for All Sites")
        return fig
    else:
        filtered_df3 = filtered_df1[filtered_df1['Launch Site'] == value1]
        fig = px.scatter(filtered_df3, x = "Payload Mass (kg)", y = "class", \
            color = "Booster Version Category", \
            title = f"Correlation between Payload and Success for {value1}")
        return fig      

# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Mar/2022 16:29:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 16:29:44] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1635179172.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 16:29:44] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_0_0m1635179172.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 16:29:44] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1635179172.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 16:29:44] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1635179172.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 16:29:44] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_0_0m1635179172.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 16:29:44] "GET /_dash-component-suites/dash/dash_table/bundle.v5_0_0m1635179172.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 16:29:44] "GET /_dash-component-suites/dash/html/dash_h